# Introduction
- In this notebook We will go through the whole process of creating a machine learning model on the  Titanic dataset, which is used by many people all over the world. It provides information on the fate of passengers on the Titanic, summarized according to economic status (class), sex, age and survival. 

- In this challenge, we are asked to predict whether a passenger on the titanic would have been survived or not.

The notebook is divided into multiple steps : 
1. Data Exploration
2. Data Preparation
3. Modeling
4. Model fine-tuning

In [21]:
# Importing all the needed libraries

# data visualization


## Data Exploration
### Data loading

In [22]:
# defining the paths to the data


In [23]:
# Loading the data into pandas dataframes


### EDA

In [24]:
# Info


In [25]:
# describe


Above we can see that 38% out of the training-set survived the Titanic. We can also see that the passenger ages range from 0.4 to 80

In [26]:
#head


### let's dig deeper ....

### What features could contribute to a high survival rate ?

#### 1. Age and Sex

#### 3. Embarked, Pclass and Sex:

#### 4.Pclass

## Data preparation

### Missing Data
#### Cabin :


#### Age :

#### Embarked:

### Feature engineering 
#### Fixing data types

#### Create age categories

In [27]:
# let's see how it's distributed


#### Creating new features
##### Age * Pclass

## Modeling

In [28]:
# Algorithms


In [29]:
# Create X_train, Y_train, X_test


#### Stochastic gradient descent (SGD) learning

In [30]:
# stochastic gradient descent (SGD) learning


#### Random forest

In [31]:
# Random Forest


#### Logistic regression 

In [32]:
# Logistic Regression


#### KNN

In [33]:
 # KNN


#### Gaussian Naive Baise

In [34]:
# Gaussian Naive Bayes


#### Other models 

In [35]:
# Perceptron


# Linear SVC


# Decision Tree


#### Comparing models 

#### K-Fold Cross Validation:

#### Feature importance 

In [36]:
# delete non relevant features


In [37]:
# Random Forest


## Model fine-tuning

In [38]:
# Random Forest
